# Practical Session 2: Introduction to Spark Dataframes and Spark SQL

In this session we will use the movielens dataset to introduce the essential features of the Spark DataFrame API and showcase its power. This tutorial also has important links to the Spark documentation and/or other relevant material.

In [0]:
import matplotlib.pyplot as plt

In [0]:
%matplotlib inline

## Downloading and unzipping the data (run only once !)

In [0]:
import urllib
import zipfile

url = 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
filehandle, _ = urllib.request.urlretrieve(url)
zip_file_object = zipfile.ZipFile(filehandle, 'r')
zip_file_object.namelist()

In [0]:
zip_file_object.extractall()

## Part 1: Spark DataFrames essentials


### Reading data

In [0]:
movies_path = "file:///databricks/driver/ml-20m/movies.csv"
ratings_path = "file:///databricks/driver/ml-20m/ratings.csv"

We read the csv files using [`spark.read`](https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html)

In [0]:
movies_df = spark.read.options(header=True).csv(movies_path)
ratings_df = spark.read.options(header=True).csv(ratings_path).sample(0.01)

We cache the read dataframes to avoid reloading them in subsequent computation.

In [0]:
movies_df.cache()
ratings_df.cache()

We then print a few rows from each dataframe.

In [0]:
movies_df.show(5)

In [0]:
ratings_df.show(5)

### Manipulating data

In [0]:
movies_df.select("title").show(5)

In [0]:
ratings_df.filter("rating=5").show(5)

In [0]:
ratings_df.groupby("userId").agg({"movieId": "count"}).show(5)

In [0]:
ratings_df.withColumn("is_rating_high", ratings_df["rating"] >= 4).show(5)

In [0]:
ratings_df.withColumn("is_rating_low", ratings_df.rating < 4).show(5)

In [0]:
ratings_df.withColumnRenamed("rating", "note").show(5)

https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions

### Built-in transformations and aggregations

In [0]:
import pyspark.sql.functions as F

In [0]:
ratings_df.select(F.avg("rating"), F.min("rating"), F.max("rating")).show()

### Joining Dataframes

In [0]:
ratings_df.join(movies_df, "movieId").show(5)

### User Defined functions (UDFs)

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import LongType

def length(string: str):
    return len(string)

length_udf = udf(length, LongType())

In [0]:
movies_df.select(length_udf("title")).show(5)

In [0]:
@udf("string")
def length2(string: str):
    return len(string)

In [0]:
movies_df.select(length2("title")).show(5)

In [0]:
title_lengths = movies_df.select(length2("title").alias("title_length"))

In [0]:
title_lengths.select(F.max("title_length")).show()

In [0]:
title_lengths.select(F.min("title_length")).show()

### Query plan inspection and caching

In [0]:
title_lengths.select(F.max("title_length")).explain()

In [0]:
title_lengths.select(F.min("title_length")).explain()

In [0]:
title_lengths.cache()
title_lengths.select(F.max("title_length")).show()

In [0]:
title_lengths.select(F.min("title_length")).explain()

### Writing csv

In [0]:
movies_df.sample(0.1).write.csv("file:///databricks/driver/ml-20m/movies-sample.csv")

In [0]:
movies_df.sample(0.1).write.mode("overwrite").csv("file:///databricks/driver/ml-20m/movies-sample.csv")

Make sure you have written on filesystem correctly. 'os.listdir' won't show you the files unless you have specified 'databricks/driver' inside write path.

In [0]:
import os
os.listdir("ml-20m")

This command writes a dataframe in parquet format :

In [0]:
ratings_parquet_path = "file:///ml-20m/ratings.parquet"
spark.read.options(header=True).csv(ratings_path).write.parquet(ratings_parquet_path)

### Question 0: Compare processing time and amount of executors used, when reading from csv versus reading from parquet, for the following pipelines:
- count total amount of records
- count total amount of records for user 1
- distinct count of timestamps

**hint** `countDistinct` method can be used for third pipeline

As we want to measure performance of parquet vs csv, we don't want caching to interfere with our measures. For this reason, we don't rely on ratings_df there.

In [0]:
spark.read.options(header=True).csv(ratings_path).count()

In [0]:
spark.read.parquet(ratings_parquet_path).count()

In [0]:
spark.read.options(header=True).csv(ratings_path).filter("userId=1").count()

In [0]:
spark.read.parquet(ratings_parquet_path).filter("userId=1").count()

In [0]:
spark.read.options(header=True).csv(ratings_path).groupby("timestamp").agg(F.count(F.lit(1))).count()

In [0]:
spark.read.parquet(ratings_parquet_path).groupby("timestamp").agg(F.count(F.lit(1))).count()

| Operation | Csv Time (s) | Parquet Time (s) | Csv Executors | Parquet Executors |
| --- | --- | --- | --- | --- |
| count | 10.8s | 1.1s | 8 | 8 |
| filter count | 24.6s | 3.6s | 8 | 8 |
| distinct count | 45.1s | 30.8s | 8 | 8 |
Parquet format contains a lots of nice optimizations !

### Question 1: Compute the (average, max, min) rating per movie, and get the highest and lowest rated movies ?

**hint** Straightforward GroupBy then Aggregate

In [0]:
import pyspark
rating_per_movie = ratings_df.groupby("movieId")\
    .agg(F.avg("rating"), F.max("rating"), F.min("rating"), F.stddev("rating"))\
    .persist(pyspark.StorageLevel.DISK_ONLY)

In [0]:
rating_per_movie.show(5)

In [0]:
movie_ratings_joined = rating_per_movie.join(movies_df, "movieId")
movie_ratings_joined.orderBy("avg(rating)").select("title", "avg(rating)").show(5, truncate=False)

In [0]:
movie_ratings_joined.orderBy(F.col("avg(rating)").desc()).select("title", "avg(rating)").show(5, truncate=False)

### Question 2: Amongst movies that were rated by at least 20 users, what are the movies with highest and lowest rating standard deviation ?

**hint** How do you use a join to keep only a subset of movies ?

In [0]:
count_per_movie = ratings_df.groupby("movieId").count()
common_movies = count_per_movie.filter("count>=20")
movie_ratings_joined_common = movie_ratings_joined.join(common_movies, "movieId")
movie_ratings_joined_common.orderBy(F.col("stddev_samp(rating)").desc()).select("title", "stddev_samp(rating)").show(5, truncate=False)

In [0]:
movie_ratings_joined_common.orderBy(F.col("stddev_samp(rating)")).select("title", "stddev_samp(rating)").show(5, truncate=False)

### Question 3: Compute the (average, max, min) rating per genre and get the highest and lowest rated genres, as well as the ones with the highest rating standard deviation ?

**hint** How can you extract the individual genres from the genres column ? How do you use a custom function to do this ?

In [0]:
movies_df_with_genres = movies_df.withColumn("genre", F.explode(F.split("genres", "\|")))
movies_df_with_genres.show(5)

In [0]:
rating_per_genre = ratings_df.join(movies_df_with_genres, "movieId") \
    .groupby("genre")\
    .agg(F.avg("rating"), F.max("rating"), F.min("rating"), F.stddev("rating"))\
    .persist(pyspark.StorageLevel.DISK_ONLY)
rating_per_genre.show(5)

In [0]:
rating_per_genre.orderBy("avg(rating)").select("genre", "avg(rating)").show(5)

In [0]:
rating_per_genre.orderBy(F.col("avg(rating)").desc()).select("genre", "avg(rating)").show(5)

In [0]:
rating_per_genre.orderBy(F.col("stddev_samp(rating)").desc()).select("genre", "stddev_samp(rating)").show(5)

### Question 4: Extract the year information from the title and compute the average rating per year (for years where more than 10 movies came out), how does the this quantity evolve ?

**hint** Extracting the year from the title can be done with a Regular Expression

In [0]:
import re
pattern = re.compile(r"\((\d+)\)")

@udf("long")
def get_year(title: str):
    found = pattern.findall(title)
    return int(found[0]) if len(found) > 0 and int(found[0])>1900 else 0

In [0]:
movie_ratings_year = movies_df\
    .withColumn("year", get_year("title"))\
    .join(ratings_df, "movieId")
avg_rating_year_pdf = movie_ratings_year\
    .filter("year>0")\
    .groupby("year").agg(F.avg("rating"), F.count("*"))\
    .toPandas()

In [0]:
avg_rating_year_pdf.set_index("year").sort_index()["avg(rating)"].plot()

### Question 5: What are the top 3 genres per year ?

**hint** Look at the answer here https://stackoverflow.com/questions/38397796/retrieve-top-n-in-each-group-of-a-dataframe-in-pyspark

In [0]:
years_with_more_than_10_movies = movie_ratings_year\
    .groupby("year").count()\
    .filter("count >= 10")\
    .select("year")

In [0]:
genre_per_year = movie_ratings_year\
    .join(movies_df_with_genres, "movieId")\
    .join(years_with_more_than_10_movies, "year")\
    .select("year", "genre")\
    .groupby("year", "genre")\
    .count()

In [0]:
genre_per_year.show(5)

In [0]:
from pyspark.sql.window import Window

window = Window.partitionBy(genre_per_year['year']).orderBy(genre_per_year['count'].desc())

genre_per_year.select('*', F.rank().over(window).alias('rank')) \
  .filter(F.col('rank') <= 3) \
  .orderBy('year') \
  .show()

### Question 6: What words of the titles cooccure the most with each genre ? Is the number of cooccurence enough ? Compute the [pointwise mutual information](https://en.wikipedia.org/wiki/Pointwise_mutual_information) between genres and movie title words, and filter out words that appear fewer than 100 times.

In [0]:
import re

@udf("array<string>")
def tokenize(string: str):
    return [w.lower() for w in re.split('\W+', string) if len(w) >= 3 and not w.isdigit()]

In [0]:
movie_words_genres = movies_df_with_genres\
    .withColumn("word", F.explode(tokenize("title")))\
    .select("word", "genre")\
    .persist(pyspark.StorageLevel.DISK_ONLY)

In [0]:
movie_words_genres.show()

In [0]:
common_words = movie_words_genres.groupby("word").count().filter("count >= 100")\
    .select("word", F.col("count").alias("word_count"))

In [0]:
cooccurences = movie_words_genres.join(common_words, "word").groupby("word", "genre").count()

In [0]:
cooccurences.show(5)

In [0]:
genre_counts = cooccurences.groupby("genre").agg(F.sum("count").alias("genre_count"))

In [0]:
total_movie_count = movies_df.count()

In [0]:
pmi = cooccurences\
    .join(genre_counts, "genre")\
    .join(common_words, "word")\
    .filter("count >= 10")\
    .withColumn("pmi", F.log((F.col("count") * F.lit(total_movie_count)) / (F.col("genre_count") * F.col("word_count"))))

In [0]:
pmi.orderBy(F.col("pmi").desc()).show()